In [1]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

import keras
from models.simple_cnn.simple_cnn import SimpleCNNContainer
import keras_tuner
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader

from torch.utils.data import DataLoader

In [ ]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

import keras
from models.simple_cnn.simple_cnn import SimpleCNNContainer
import keras_tuner
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader

from torch.utils.data import DataLoader

## Let's try Grayscale
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)

# todo change name of KFoldController and also check it is shuffled
# I am using the KFoldController class to simply get 20% of the dataset as a validation fold. I won't do any
# k-Fold CV on the parameters search or else this will take forever
dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32)
validation_dataloader = DataLoader(dataset=validation, batch_size=32)
hypermodel = SimpleCNNContainer((3, 224, 224))
# We are learning the structure of the network with fixed parameters for learning.
# As we know hyperparamters of NN are: Neurons, Layers, Optimizer, Learning-Rate, Epochs, Batch size etc
hyperparameters.Fixed("lr", value=1e-3)

## TODO VEDI QUESTI PARAMETERI https://keras.io/api/keras_tuner/tuners/base_tuner/#tuner-class
tuner = keras_tuner.Hyperband(
    hypermodel,
    hyperparameters=hyperparameters,
    hyperband_iterations=3,
    objective='val_loss',
    tune_new_entries=True,
    overwrite=True,
    directory="cnn_search",
    project_name="standard_image"  # should be highest possible
)

tuner.search_space_summary()
tuner.search(train_dataloader, epochs=30, validation_data=validation_dataloader,
             callbacks=[
                 # keras.callbacks.TensorBoard("/tmp/tb_logs"),
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
             ])

In [ ]:
tuner.get_best_hyperparameters()

In [ ]:
tuner.get_best_hyperparameters()

In [ ]:
tuner.results_summary()

In [ ]:
tuner.get_best_hyperparameters()

In [2]:
from keras_tuner import HyperParameters

## Lets try our best hypermodel
hyperparameters = HyperParameters()
hyperparameters.Fixed("lr", value=1e-3)
hyperparameters.Fixed(name=f"kernel_0", value=3),
hyperparameters.Fixed(name=f"filters_0", value=32)

hyperparameters.Fixed(name=f"kernel_1", value=3),
hyperparameters.Fixed(name=f"filters_1", value=64)
hyperparameters.Fixed(name="hidden_units", value=64)

64

In [3]:
# todo: trova l'errore. Se uso il KFOLD pare funzionare correttamrente l'apprendimento mentre con i dataset diretti no. Che sia un problema di shuffle? Che sia che i channel non sono invertiti?
hypermodel = SimpleCNNContainer((3, 224, 224))

train, test = dataset_loader((224, 224), is_grayscale=False)

dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32)
validation_dataloader = DataLoader(dataset=validation, batch_size=32)

model = hypermodel.build(hyperparameters)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_cnn (InputLayer)         │ (None, 3, 224, 224)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 224, 224)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 112, 112)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 112, 112)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 56, 56)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    12,845,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,864,577 (49.07 MB)

 Trainable params: 12,864,577 (49.07 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
i_history = model.fit(train_dataloader, epochs=100, validation_data=validation_dataloader,
                      callbacks=[
                          # To avoid keep going
                          keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                                        restore_best_weights=True),

                          # To reduce learning rate (like Hot Stocastic bla bla)
                          #keras.callbacks.LearningRateScheduler(
                          #    learning_rate_reschedule.learning_rate_reschedule)
                      ])

Epoch 1/100
  1/119 ━━━━━━━━━━━━━━━━━━━━ 45s 386ms/step - accuracy: 0.3750 - loss: 0.7017

/home/jacopo/miniconda3/envs/keras-pytorch/lib/python3.12/site-packages/keras/src/backend/torch/nn.py:412: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1704987296916/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  outputs = tnn.conv2d(


119/119 ━━━━━━━━━━━━━━━━━━━━ 45s 376ms/step - accuracy: 0.5304 - loss: 0.6874 - val_accuracy: 0.5470 - val_loss: 0.6754
Epoch 2/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 44s 373ms/step - accuracy: 0.5983 - loss: 0.6685 - val_accuracy: 0.6093 - val_loss: 0.6598
Epoch 3/100
 66/119 ━━━━━━━━━━━━━━━━━━━━ 16s 303ms/step - accuracy: 0.6618 - loss: 0.6532

KeyboardInterrupt: 